In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 32,
            'batch_size_train': 20,
            'batch_size_memory': 20,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.00
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=MNIST/seed=1_epoch=1_lr=0.001_alpha=0.0_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]


In [3]:
from algorithms.gss import GSSGreedy
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10).to(params['device'])
algorithm = GSSGreedy(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
from trainers.baselines import BaseMemoryContinualTrainer

trainer = BaseMemoryContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 99.89795918367346, 'loss': 2.267684800722075e-05, 'std': 0.10204081632653184}
---------------------------- Task 2 -----------------------
[2] Eval metrics for task 1 >> {'accuracy': 93.51411489706014, 'loss': 0.00116691486491661, 'std': 0.7590128562438181}
[2] Eval metrics for task 2 >> {'accuracy': 97.30994320362268, 'loss': 0.000292949172340691, 'std': 0.3138191726149375}
---------------------------- Task 3 -----------------------
[3] Eval metrics for task 1 >> {'accuracy': 91.08356558482424, 'loss': 0.0012448886396191645, 'std': 4.24683089094669}
[3] Eval metrics for task 2 >> {'accuracy': 54.92775731061479, 'loss': 0.008067858674500527, 'std': 25.56729219433571}
[3] Eval metrics for task 3 >> {'accuracy': 98.34920953850931, 'loss': 0.00019870428000691098, 'std': 1.0397924981505657}
---------------------------- Task 4 -----------------------
[4] Eval metrics for task 1 >> {'accura

In [13]:
tid = 2
loader = torch.utils.data.DataLoader(torch.utils.data.Subset(benchmark.trains[tid], benchmark.memory_indices_train[tid]), batch_size=32)
for items in loader:
    img, target, task_id, index, sample_weight = items
    break

target


tensor([3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 2, 2, 3, 3, 3, 3])

In [6]:
metric_manager_callback.meters['std'].get_std()

[0.10204081632653184,
 1.9847845292996977,
 24.184163967404114,
 24.043098010088496,
 22.754890378265582]

In [7]:
metric_manager_callback.meters['std'].get_data()

array([ 0.102,  1.985, 24.184, 24.043, 22.755])

In [14]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.898,  0.   ,  0.   ,  0.   ,  0.   ],
       [93.514, 97.31 ,  0.   ,  0.   ,  0.   ],
       [91.084, 54.928, 98.349,  0.   ,  0.   ],
       [88.832, 50.381, 77.815, 98.356,  0.   ],
       [83.063, 43.93 , 63.817, 83.014, 96.262]])

In [9]:
metric_manager_callback.meters['accuracy'].compute_overall()

85.92525532391896

In [10]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"fairness:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:85.92525532391896
fairness:14.613795540276886


In [11]:
metric_manager_callback.meters['std'].compute_overall()

14.613795540276886

In [15]:
metric_manager_callback.meters['accuracy']